In [10]:
import requests
from bs4 import BeautifulSoup as bs

url ="https://www.yes24.com/24/category/bestseller"
payload = dict(CategoryNumber="001",sumgb="06",PageNumber="{IdxNum}")

response = requests.get(url,params=payload)

soup = bs(response.content,"lxml")
response.url


'https://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=06&PageNumber=1'

In [37]:


data_list = []  # 데이터를 저장할 리스트 초기화
for i in soup.select("#category_layout"):
    notices = i.select("tr:nth-child(2n)")  # 공지 데이터
    books = i.select("tr:nth-child(2n+1)")  # 도서 정보 데이터

    for notice, book in zip(notices, books):
        star_on_count = 0
        star_off_count = 0

        book_data = {
            "구분": book.select_one(".goodsTxtInfo").select_one("p").text[0:5],
            "제목": book.select_one(".goodsTxtInfo p").select_one("a").text,
            "작가": book.select_one(".aupu").text.split("|")[0].strip(),
            "출판사": book.select_one(".aupu").text.split("|")[1].strip(),
            "정가": book.select_one(".goodsTxtInfo p:nth-child(3)").text.split("원")[0],
            "할인가" : book.select_one(".goodsTxtInfo p:nth-child(3)").text.split("원")[1].replace("→","").strip(),
            "출간일" : book.select_one(".aupu").text.split("|")[2].strip(),
            "별점": "",
            "편집구성 별점": ""
        }

        # 도서 정보의 별점 데이터 처리
        if book.select_one(".review") is not None:
            for rate in book.select_one(".review").select("img")[0:5]:
                src = rate["src"]
                if "StarOn.gif" in src:
                    star_on_count += 1
                elif "StarOff.gif" in src:
                    star_off_count += 1
            total_star_count = star_on_count + star_off_count
            if total_star_count > 0:
                rating = (star_on_count / total_star_count) * 5
            else:
                rating = 0
            book_data["별점"] = rating

        # 편집구성 별점 데이터 처리 (이하 동일)
        if book.select_one(".review") is not None:
            for rate in book.select_one(".review").select("img")[5:11]:
                src = rate["src"]
                if "StarOn.gif" in src:
                    star_on_count += 1
                elif "StarOff.gif" in src:
                    star_off_count += 1
            total_star_count = star_on_count + star_off_count
            if total_star_count > 0:
                rating = (star_on_count / total_star_count) * 5
            else:
                rating = 0
            book_data["편집구성 별점"] = rating

        # 공지 데이터 처리
        notice_data = {
            "상세내용": notice.select_one(".read").text.strip(),
        }

        if notice_data:
            data_list.append({**notice_data, **book_data})
        else:
            data_list.append(book_data)
        if IdxNum >= 50:
            break





# 데이터 리스트 출력 (테스트용)
for data in data_list:
    print(data)


{'상세내용': '첫 발표 이후 43년, 마음에 품어왔던 소설을 마침내 완성하다. 하루키적 상상력의 모든 것이 담긴 결정적 세계!  “이 작품에는 무언가 나에게 매우 중요한 요소가 포함되어 있다고, 처음부터 그렇게 느껴왔다.” _무라카미 하루키   무라카미 하루키의 신작 장편소설 『도시와 그 불확실한 벽』의 한국어판이 9월 6...', '구분': '[도서] ', '제목': '도시와 그 불확실한 벽', '작가': '무라카미 하루키 저/홍은주 역', '출판사': '문학동네', '정가': '19,500', '할인가': '17,550', '출간일': '2023년 09월', '별점': 4.0, '편집구성 별점': 4.0}
{'상세내용': '‘미래의 나’를 적용하는 과학 분야 세계 최고의 전문가, 자기계발 분야 파워블로거이자 베스트셀러 작가의 신작 세계적인 동기부여 전문가, 토니 로빈스 강력 추천!  현재와 미래를 더 가치 있게 바꾸고 싶다면, ‘미래의 나’와 연결하라!  살 날이 얼마 남지 않은 ‘미래의 내’가 현재로 시간 여행을 왔다고 상...', '구분': '[도서] ', '제목': '퓨처 셀프', '작가': '벤저민 하디 저/최은아 역', '출판사': '상상스퀘어', '정가': '19,800', '할인가': '17,820', '출간일': '2023년 08월', '별점': 5.0, '편집구성 별점': 5.0}
{'상세내용': '“등에 화살이 꽂힌 채 길 없는 길을 묵묵히 걸어가겠다.” 법대 교수, 법무부장관에서 자연인으로 돌아온 조국의 첫 에세이  2023년 6월 12일 법학자 조국은 서울대로부터 교수직을 파면당했다. 이 책은 법대 교수 조국이 법을 공부한 이유와 자신이 생각하는 공부의 참된 의미에 대해 기록한 책 『왜 나는 법을 공부하...', '구분': '[도서] ', '제목': '디케의 눈물', '작가': '조국 저', '출판사': '다산북스', '정가': '18,800', '할인가': '16,920', '출간일': '2023년 08월', '별점': 4.0, 

In [5]:
import requests
from bs4 import BeautifulSoup as bs
import time

data_list = []  # 데이터를 저장할 리스트 초기화
IdxNum = 1

while True:  # 무한 루프로 설정

    url = "https://www.yes24.com/24/category/bestseller"
    payload = dict(CategoryNumber="001", sumgb="06", PageNumber=f"{IdxNum}")

    response = requests.get(url, params=payload)

    soup = bs(response.content, "lxml")

    for i in soup.select("#category_layout"):
        notices = i.select("tr:nth-child(2n)")  # 공지 데이터
        books = i.select("tr:nth-child(2n+1)")  # 도서 정보 데이터

        for notice, book in zip(notices, books):
            star_on_count = 0
            star_off_count = 0
            print(book.select_one(".goodsTxtInfo .review"))
            book_data = {
                "구분": book.select_one(".goodsTxtInfo").select_one("p").text[0:5],
                "제목": book.select_one(".goodsTxtInfo p").select_one("a").text,
                "작가": book.select_one(".aupu").text.split("|")[0].strip(),
                "출판사": book.select_one(".aupu").text.split("|")[1].strip(),
                "정가": book.select_one(".goodsTxtInfo p:nth-child(3)").text.split("원")[0],
                "할인가" : book.select_one(".goodsTxtInfo p:nth-child(3)").text.split("원")[1].replace("→","").strip(),
                "출간일" : book.select_one(".aupu").text.split("|")[2].strip(),
                "별점": "",
                "편집구성 별점": ""
            }


            if book.select_one(".review") is not None:
                for rate in book.select_one(".review").select("img")[0:5]:
                    src = rate["src"]
                    if "StarOn.gif" in src:
                        star_on_count += 1
                    elif "StarOff.gif" in src:
                        star_off_count += 1
                total_star_count = star_on_count + star_off_count
                if total_star_count > 0:
                    rating = (star_on_count / total_star_count) * 5
                else:
                    rating = 0
                book_data["별점"] = rating


            if book.select_one(".review") is not None:
                for rate in book.select_one(".review").select("img")[5:11]:
                    src = rate["src"]
                    if "StarOn.gif" in src:
                        star_on_count += 1
                    elif "StarOff.gif" in src:
                        star_off_count += 1
                total_star_count = star_on_count + star_off_count
                if total_star_count > 0:
                    rating = (star_on_count / total_star_count) * 5
                else:
                    rating = 0
                book_data["편집구성 별점"] = rating

            # 공지 데이터 처리
            notice_data = {
                "상세내용": notice.select_one(".read").text.strip() if notice.select_one(".read") else "None",
            }

            if notice_data:
                data_list.append({**notice_data, **book_data})
            else:
                data_list.append(book_data)

    # 내부 루프 외부에서 IdxNum 증가
    IdxNum += 1
    print(f"총{IdxNum}출력중")

    # 페이지 수가 50 페이지일 때 루프 종료
    if IdxNum > 50:
        break
    
    # 페이지 크롤링 후 일정 시간 쉬기
    time.sleep(5)

# 데이터 리스트 출력 (테스트용)
for data in data_list:
    print(data)


<p class="review">회원리뷰 (<a href="/Product/Goods/122090075?reviewYn=Y">21개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/> | 편집구성 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/>
</p>
<p class="review">회원리뷰 (<a href="/Product/Goods/122090360?reviewYn=Y">2개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.y

<p class="review">회원리뷰 (<a href="/Product/Goods/118104953?reviewYn=Y">77개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/> | 편집구성 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/>
</p>
<p class="review">회원리뷰 (<a href="/Product/Goods/121992615?reviewYn=Y">71개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.

<p class="review">회원리뷰 (<a href="/Product/Goods/116206662?reviewYn=Y">17개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/> | 편집구성 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/>
</p>
<p class="review">회원리뷰 (<a href="/Product/Goods/121772149?reviewYn=Y">7개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.ye

<p class="review">회원리뷰 (<a href="/Product/Goods/8759796?reviewYn=Y">251개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/> | 편집구성 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/>
</p>
<p class="review">회원리뷰 (<a href="/Product/Goods/117643865?reviewYn=Y">47개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.y

<p class="review">회원리뷰 (<a href="/Product/Goods/119512317?reviewYn=Y">19개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/> | 편집구성 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/>
</p>
<p class="review">회원리뷰 (<a href="/Product/Goods/97255028?reviewYn=Y">365개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.

...

<p class="review">회원리뷰 (<a href="/Product/Goods/122280618?reviewYn=Y">13개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/> | 편집구성 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/>
</p>
None
<p class="review">회원리뷰 (<a href="/Product/Goods/121924953?reviewYn=Y">65개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://i

<p class="review">회원리뷰 (<a href="/Product/Goods/105981214?reviewYn=Y">38개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/> | 편집구성 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/>
</p>
<p class="review">회원리뷰 (<a href="/Product/Goods/102818880?reviewYn=Y">34개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.

<p class="review">회원리뷰 (<a href="/Product/Goods/117030338?reviewYn=Y">114개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/> | 편집구성 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/>
</p>
<p class="review">회원리뷰 (<a href="/Product/Goods/116046489?reviewYn=Y">57개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image

<p class="review">회원리뷰 (<a href="/Product/Goods/3334740?reviewYn=Y">20개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/> | 편집구성 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/>
</p>
None
None
<p class="review">회원리뷰 (<a href="/Product/Goods/122426959?reviewYn=Y">5개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https:/

<p class="review">회원리뷰 (<a href="/Product/Goods/115833661?reviewYn=Y">12개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/> | 편집구성 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOff.gif"/>
</p>
<p class="review">회원리뷰 (<a href="/Product/Goods/114299272?reviewYn=Y">15개</a>) | 내용 <img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.yes24.com/sysimage/interface/StarOn.gif"/><img src="https://image.

In [38]:
data_list

[{'상세내용': '첫 발표 이후 43년, 마음에 품어왔던 소설을 마침내 완성하다. 하루키적 상상력의 모든 것이 담긴 결정적 세계!  “이 작품에는 무언가 나에게 매우 중요한 요소가 포함되어 있다고, 처음부터 그렇게 느껴왔다.” _무라카미 하루키   무라카미 하루키의 신작 장편소설 『도시와 그 불확실한 벽』의 한국어판이 9월 6...',
  '구분': '[도서] ',
  '제목': '도시와 그 불확실한 벽',
  '작가': '무라카미 하루키 저/홍은주 역',
  '출판사': '문학동네',
  '정가': '19,500',
  '할인가': '17,550',
  '출간일': '2023년 09월',
  '별점': 4.0,
  '편집구성 별점': 4.0},
 {'상세내용': '‘미래의 나’를 적용하는 과학 분야 세계 최고의 전문가, 자기계발 분야 파워블로거이자 베스트셀러 작가의 신작 세계적인 동기부여 전문가, 토니 로빈스 강력 추천!  현재와 미래를 더 가치 있게 바꾸고 싶다면, ‘미래의 나’와 연결하라!  살 날이 얼마 남지 않은 ‘미래의 내’가 현재로 시간 여행을 왔다고 상...',
  '구분': '[도서] ',
  '제목': '퓨처 셀프',
  '작가': '벤저민 하디 저/최은아 역',
  '출판사': '상상스퀘어',
  '정가': '19,800',
  '할인가': '17,820',
  '출간일': '2023년 08월',
  '별점': 5.0,
  '편집구성 별점': 5.0},
 {'상세내용': '“등에 화살이 꽂힌 채 길 없는 길을 묵묵히 걸어가겠다.” 법대 교수, 법무부장관에서 자연인으로 돌아온 조국의 첫 에세이  2023년 6월 12일 법학자 조국은 서울대로부터 교수직을 파면당했다. 이 책은 법대 교수 조국이 법을 공부한 이유와 자신이 생각하는 공부의 참된 의미에 대해 기록한 책 『왜 나는 법을 공부하...',
  '구분': '[도서] ',
  '제목': '디케의 눈물',
  '작가': '조국 저',
  '출판사': '다산북스',
  '정가': '18,800

In [3]:
import pandas as pd 

df = pd.DataFrame(data_list)
df

,상세내용,구분,제목,작가,출판사,정가,할인가,출간일,별점,편집구성 별점
0,"첫 발표 이후 43년, 마음에 품어왔던 소설을 마침내 완성하다. 하루키적 상상력의 ...",[도서],도시와 그 불확실한 벽,무라카미 하루키 저/홍은주 역,문학동네,"19,500","17,550",2023년 09월,4.0,4.0
1,"‘미래의 나’를 적용하는 과학 분야 세계 최고의 전문가, 자기계발 분야 파워블로거이...",[도서],퓨처 셀프,벤저민 하디 저/최은아 역,상상스퀘어,"19,800","17,820",2023년 08월,5.0,5.0
2,"“등에 화살이 꽂힌 채 길 없는 길을 묵묵히 걸어가겠다.” 법대 교수, 법무부장관에...",[도서],디케의 눈물,조국 저,다산북스,"18,800","16,920",2023년 08월,4.0,4.0
3,유튜브 261만 구독 돌파! 인기 크리에이터 흔한남매 웃음에 겨워 춤이 절로 나오는...,[도서],흔한남매 14,흔한남매 원저/백난도 글/유난희 그림/흔한컴퍼니 감수,미래엔아이세움,"14,500","13,050",2023년 09월,,
4,"재야의 명저 『세이노의 가르침』 2023년판 정식 출간! 순자산 천억 원대 자산가,...",[도서],세이노의 가르침,세이노(SayNo) 저,데이원,"7,200","6,480",2023년 03월,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...
964,세계 역사가 한 편의 역사 영화처럼 보인다! 한 편의 영화처럼 읽는 세계사 속 전쟁...,[도서],썬킴의 거침없는 세계사,썬킴 저,지식의숲,"16,500","14,850",2021년 04월,4.0,4.0
965,명작은 영원하다! 모두 이노우에 타케히코가 새로 작업한 컬러 일러스트로 만들어진 신...,[만화],슬램덩크 신장재편판 16,이노우에 타케히코 글그림,대원,"7,000","6,300",2018년 11월,4.0,4.0
966,판타스틱 드래곤 월드 무시무시하게만 보였던 포이즌 드래곤의 반전 매력?! 롤랜드왕...,[도서],드래곤 마스터 5 포이즌 드래곤의 노래,트레이시 웨스트 글/데미안 존스 그림/윤영 역,다산어린이,"13,000","11,700",2023년 09월,,
967,"이 교재는 지금까지의 출제경향을 철저하게 분석하였고, 특히 2022/23년 주요 승...",[도서],2024 정태정 해양경찰학개론 실전동형모의고사,정태정 편저,멘토링,"29,000","26,100",2023년 09월,,


In [4]:
df.to_csv('예스24.csv',encoding='utf-8')